## Занятие 4. Разработка ml проектов: хорошие и плохие практики
На занятии 3 мы написали пайплайн обучения семантического сегментатора подводных фото в одном jupyter ноутбуке.

На этом занятии мы оформим этот пайплайн в ml проект так, чтобы его можно было поддерживать и развивать.


In [1]:
from pathlib import Path
ROOT_PATH = Path().absolute()
assert ROOT_PATH.name == 'c04', ROOT_PATH.name
ROOT_PATH

PosixPath('/home/alexander/computerScience/phystech/9sem/abbyy/course_cvdl/classes/c04')

### Плохая практика 1: использование jupyter ноутбуков в качестве основного средства разработки
- https://www.kdnuggets.com/2019/11/notebook-anti-pattern.html
- https://analyticsindiamag.com/an-argument-against-using-jupyter-notebook-for-machine-learning/
- https://medium.com/skyline-ai/jupyter-notebook-is-the-cancer-of-ml-engineering-70b98685ee71

Главные проблемы:
- нелинейность исполнения кода (сложно читать и сложно воспроизводить)
- невозможность тестирования
- нечитыемые diff-ы в git (сложно разобраться в истории и работать совместно)

Jupyter ноутбуки хорошо подходят чтобы:
- визуализировать данные
- набросать прототип
- поделиться однократным результатом


#### **Решение**: 
Использовать стандартные инструменты разработки ЯП, например - Python пакеты.

In [2]:
# Python ищет пакеты в PYTHONPATH, если пакет не найден - то получаем ModuleNotFound
from suim_segmentation.data import SuimDataset

In [3]:
cd package

/home/alexander/computerScience/phystech/9sem/abbyy/course_cvdl/classes/c04/package


In [3]:
# В PYTHONPATH всегда содержится current_dir, так что из родительской директории можно импортировать пакет
from suim_segmentation.data import SuimDataset

In [4]:
from suim_segmentation import data as suim_data
suim_data.__file__

'/home/alexander/computerScience/phystech/9sem/abbyy/course_cvdl/classes/c04/package/suim_segmentation/data.py'

In [5]:
cd ..

/home/alexander/computerScience/phystech/9sem/abbyy/course_cvdl/classes


Если пакет находится в текущей папке, то его можно импортировать.

Это не очень удобно - код можно будет вызывать только при определенной текущей директории.

**Правильное решение - сделать пакет устанавливаемым!**

### 1.1 Пакетируем код

На сегодня (2022 год) рекомендуется пакетировать код с помощью `pyproject.toml` файла:
- поддерживает разные системы сборки (не только setuptools)
- поддерживает C++ расширения
- стандартизирован в PEP-518, PEP-621

Есть и другие [способы](https://packaging.python.org/en/latest/glossary/?highlight=setup.py#term-setup.py), которые пока более распространены, но `pyproject.toml` является рекомендуемым

In [7]:
ls package

data/  pyproject.toml  suim_segmentation/  suim_segmentation.egg-info/


In [8]:
%%writefile {str(ROOT_PATH / 'package' / 'pyproject.toml')}

[project]
name = "suim_segmentation"
description = "ML project example package"
version = "0.1.0"

Overwriting /home/alexander/computerScience/phystech/9sem/abbyy/course_cvdl/classes/c04/package/pyproject.toml


### Задачка 1
Пользуясь гайдом https://packaging.python.org/en/latest/tutorials/packaging-projects/#creating-pyproject-toml, заполните pyproject.toml так, чтобы пакет можно было установить.

*Возможно, нужно будет обновить pip: `python -m pip install -U pip`*

c04.ipynb  data/  dvc_scheme.png  package/  README.md


In [12]:
! pip install -e ./package/

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Obtaining file:///home/alexander/computerScience/phystech/9sem/abbyy/course_cvdl/classes/c04/package
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build editable did not run successfully.
  │ exit code: 1
  ╰─> [14 lines of output]
      error: Multiple top-level packages discovered in a flat-layout: ['data', 'suim_segmentation'].
      
      To avoid accidental inclusion of unwanted files or directories,
      setuptools will not proceed with this build.
      
      If you are trying to create a single distribution with multiple packages
      on purpose, you should not rely on automatic discovery.
      Instead, consider the following options:
      
      1. set up custom discovery (`find` directive with `include` or `exclude`)
      2.

Пакеты c `pyproject.toml` (или `setup.py`) можно устанавливать через git:
`TODO`

### 1.1 Проверяем код на PEP-8 с помощью `pylint`
Pylint - инструмент для проверки кода на соответствие PEP-8.

`pip install pylint`

Вывод всех нарушений PEP8 и оценки вашего кода: `python -m pylint suim_segmentation`

Вывод только ошибок: `python -m pylint suim_segmentation/ -E`

In [13]:
cd package/

/home/alexander/computerScience/phystech/9sem/abbyy/course_cvdl/classes/c04/package


In [14]:
! python -m pylint suim_segmentation

************* Module suim_segmentation.trainer
suim_segmentation/trainer.py:1:0: C0114: Missing module docstring (missing-module-docstring)
suim_segmentation/trainer.py:9:0: C0115: Missing class docstring (missing-class-docstring)
suim_segmentation/trainer.py:9:0: R0902: Too many instance attributes (8/7) (too-many-instance-attributes)
suim_segmentation/trainer.py:32:4: C0116: Missing function or method docstring (missing-function-docstring)
suim_segmentation/trainer.py:38:21: C0103: Variable name "y" doesn't conform to snake_case naming style (invalid-name)
suim_segmentation/trainer.py:39:15: C0103: Variable name "y" doesn't conform to snake_case naming style (invalid-name)
suim_segmentation/trainer.py:40:12: C0103: Variable name "yp" doesn't conform to snake_case naming style (invalid-name)
suim_segmentation/trainer.py:60:12: C0103: Variable name "e" doesn't conform to snake_case naming style (invalid-name)
suim_segmentation/trainer.py:61:18: W1309: Using an f-string that does not ha

### 1.2 Форматируем код с помощью `black`
Часть ошибок форматирования можно поправить автоматически с помощью black.
`pip install black`

Форматирование: `python -m black suim_segmentation/`

In [15]:
! python -m black suim_segmentation/

reformatted suim_segmentation/run.py

All done! ✨ 🍰 ✨
1 file reformatted, 6 files left unchanged.


In [16]:
! python -m pylint suim_segmentation

************* Module suim_segmentation.trainer
suim_segmentation/trainer.py:1:0: C0114: Missing module docstring (missing-module-docstring)
suim_segmentation/trainer.py:9:0: C0115: Missing class docstring (missing-class-docstring)
suim_segmentation/trainer.py:9:0: R0902: Too many instance attributes (8/7) (too-many-instance-attributes)
suim_segmentation/trainer.py:32:4: C0116: Missing function or method docstring (missing-function-docstring)
suim_segmentation/trainer.py:38:21: C0103: Variable name "y" doesn't conform to snake_case naming style (invalid-name)
suim_segmentation/trainer.py:39:15: C0103: Variable name "y" doesn't conform to snake_case naming style (invalid-name)
suim_segmentation/trainer.py:40:12: C0103: Variable name "yp" doesn't conform to snake_case naming style (invalid-name)
suim_segmentation/trainer.py:60:12: C0103: Variable name "e" doesn't conform to snake_case naming style (invalid-name)
suim_segmentation/trainer.py:61:18: W1309: Using an f-string that does not ha

### 1.3 Тюним правил PEP-8 под свой проект
Для pylint можно создать набор правил, по умолчанию - файл `.pylintrc`

In [17]:
str(ROOT_PATH / 'package' / '.pylintrc')

'/home/alexander/computerScience/phystech/9sem/abbyy/course_cvdl/classes/c04/package/.pylintrc'

In [18]:
%%writefile {str(ROOT_PATH / 'package' / '.pylintrc')}

[TYPECHECK]
# ignore torch member warnings - they are broken
generated-members=numpy.*, torch.*

[FORMAT]
# allow x as a good name
good-names=i,j,k,x
# increase max length
max-line-length=128

Overwriting /home/alexander/computerScience/phystech/9sem/abbyy/course_cvdl/classes/c04/package/.pylintrc


In [19]:
! python -m pylint suim_segmentation

************* Module suim_segmentation.trainer
suim_segmentation/trainer.py:1:0: C0114: Missing module docstring (missing-module-docstring)
suim_segmentation/trainer.py:9:0: C0115: Missing class docstring (missing-class-docstring)
suim_segmentation/trainer.py:9:0: R0902: Too many instance attributes (8/7) (too-many-instance-attributes)
suim_segmentation/trainer.py:32:4: C0116: Missing function or method docstring (missing-function-docstring)
suim_segmentation/trainer.py:38:21: C0103: Variable name "y" doesn't conform to snake_case naming style (invalid-name)
suim_segmentation/trainer.py:39:15: C0103: Variable name "y" doesn't conform to snake_case naming style (invalid-name)
suim_segmentation/trainer.py:40:12: C0103: Variable name "yp" doesn't conform to snake_case naming style (invalid-name)
suim_segmentation/trainer.py:60:12: C0103: Variable name "e" doesn't conform to snake_case naming style (invalid-name)
suim_segmentation/trainer.py:61:18: W1309: Using an f-string that does not ha

### 1.4 Пишем "точку входа" в пайплайн (скрипт обучения) 

In [24]:
%%writefile {str(ROOT_PATH / 'package' / 'suim_segmentation' / 'run.py')}

import argparse
from pathlib import Path
import wandb

import torch
from torch.utils import data as tdata
from tqdm import tqdm

from .data import SuimDataset, EveryNthFilterSampler
from .model import SuimModel
from .loss import DiceLoss
from .metrics import Accuracy
from .trainer import Trainer


def run_pipeline(args):
    wandb.init(project='test1', config=vars(args))
    device = args.device
    model = SuimModel().to(device)
    opt = torch.optim.Adam(model.parameters(), lr=args.lr)

    train_val_ds = SuimDataset(Path(args.train_data), masks_as_color=False, target_size=(256, 256))
    test_ds = SuimDataset(Path(args.test_data), masks_as_color=False, target_size=(256, 256))
    
    test_iter = tdata.DataLoader(test_ds, batch_size=args.batch_size, shuffle=False)
    train_iter = tdata.DataLoader(train_val_ds, batch_size=args.batch_size, 
        sampler=EveryNthFilterSampler(dataset_size=len(train_val_ds), n=5, pass_every_nth=False, shuffle=True)
    )
    val_iter = tdata.DataLoader(train_val_ds, batch_size=args.batch_size, 
        sampler=EveryNthFilterSampler(dataset_size=len(train_val_ds), n=5, pass_every_nth=True, shuffle=False)
    )
    
    loss = DiceLoss()
    metric = Accuracy()
    
    trainer = Trainer(
        net=model,
        opt=opt,
        train_loader=train_iter,
        val_loader=val_iter,
        test_loader=test_iter,
        loss=loss,
        metric=metric,
    )
    
    mean = lambda values: sum(values) / len(values)
    
    for e in range(args.num_epochs):
        print(f"Epoch {e}")
        with_testing = (e == args.num_epochs - 1)
        epoch_stats = trainer(num_epochs=1, with_testing=with_testing)
        train_loss, train_metric = epoch_stats['train'][0]
        val_loss, val_metric = epoch_stats['val'][0]
        wandb.log({"train": {"loss": train_loss, "metric": train_metric}, 
                   "val": {"loss": val_loss, "acc": val_metric}}, step=e)
        assert isinstance(train_loss, list), type(train_loss)
    
    if args.num_epochs > 0:
        test_loss, test_metric = epoch_stats['test'][0]
        wandb.run.summary['test.loss'] = mean(test_loss)
        wandb.run.summary['test.metric'] = mean(test_metric)

    wandb.run.summary['haha'] = 'hehe'
    wandb.finish()

def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--name", type=str, required=True)
    parser.add_argument("--train-data", type=str, required=True)
    parser.add_argument("--test-data", type=str, required=True)
    parser.add_argument("--lr", type=float, required=True)
    parser.add_argument("--num-epochs", type=int, required=True)
    parser.add_argument("--batch-size", type=int, default=16)
    parser.add_argument("--device", type=str, default='cpu:0')
    return parser.parse_args()


if __name__ == "__main__":
    args = parse_args()
    run_pipeline(args)
    print("Finished")

Overwriting /home/alexander/computerScience/phystech/9sem/abbyy/course_cvdl/classes/c04/package/suim_segmentation/run.py


### Задачка 2
Пользуясь ноутбуком с предыдущего занятия, допишите `suim_segmentation/run.py`, чтобы тренировка корректно запускалась.


In [25]:
! python -m suim_segmentation.run --name=baseline --lr=0.03 --num-epochs=0 --batch-size=32 --device=cuda:0 \
    --train-data=/home/alexander/computerScience/phystech/9sem/abbyy/course_cvdl/data/train_val \
    --test-data=/home/alexander/computerScience/phystech/9sem/abbyy/course_cvdl/data/TEST

wandb: Currently logged in as: vashchilkoav. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.13.4
wandb: Run data is saved locally in /home/alexander/computerScience/phystech/9sem/abbyy/course_cvdl/classes/c04/package/wandb/run-20221010_163943-2eqkse4p
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run vibrant-music-2
wandb: ⭐️ View project at https://wandb.ai/vashchilkoav/test1
wandb: 🚀 View run at https://wandb.ai/vashchilkoav/test1/runs/2eqkse4p
wandb: Waiting for W&B process to finish... (success).
wandb: \ 0.001 MB of 0.026 MB uploaded (0.000 MB deduped)
wandb: Run summary:
wandb: haha hehe
wandb: 
wandb: Synced vibrant-music-2: https://wandb.ai/vashchilkoav/test1/runs/2eqkse4p
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20221010_163943-2eqkse4p/logs
Finished


### Плохая практика 2: "ручное" копирование данных

*Из с03/README.md:*

`скачать датасет SUIM с GoogleDrive: https://drive.google.com/drive/folders/10KMK0rNB43V2g30NcA1RYipL535DuZ-h`

- В реальных проектах (индустрия/соревнования) часто используется несколько датасетов, а не один
- Они могут быть в разных форматах, а код проекта обычно работает только с одним форматом
- Датасет может обновляться - даже у ImageNet есть [version2](https://proceedings.mlr.press/v97/recht19a/recht19a.pdf)
- Часто датасет нужен на разных машинах (например, локальной и devbox с GPU)

#### **Решение**: 
Использовать инструменты для управления данными, например - [dvc](https://dvc.org)

![dvc scheme was not loaded](dvc_scheme.png "DVC scheme")

### 2.1 Добавляем google-drive в качестве хранилища

https://dvc.org/doc/user-guide/how-to/setup-google-drive-remote

In [21]:
cd ..

/home/alexander/computerScience/phystech/9sem/abbyy/course_cvdl


In [22]:
! dvc init --subdir

Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


In [23]:
! dvc remote add mygoogledrive https://drive.google.com/drive/folders/177I0TiGEaqGXB9L1waDr8Lh1UI7IyVk1?usp=sharing

In [24]:
! dvc remote list

mygoogledrive	https://drive.google.com/drive/folders/177I0TiGEaqGXB9L1waDr8Lh1UI7IyVk1?usp=sharing


Добавим test-данные в индекс dvc

In [62]:
import shutil

source_dataset = ROOT_PATH.parent.parent / 'data' / 'TEST'
assert source_dataset.exists()

target_dataset = ROOT_PATH / 'data'/ 'test'
shutil.copytree(str(source_dataset), str(target_dataset))

'/home/alexander/computerScience/phystech/9sem/abbyy/course_cvdl/classes/c04/data/test'

In [25]:
cd data

/home/alexander/computerScience/phystech/9sem/abbyy/course_cvdl/data


In [26]:
! dvc add TEST

⠋ Checking graph                                                   ⠋ Checking graph
Adding...                                                                       
!
Building data objects from TEST                       |0.00 [00:00,      ?obj/s]
Building data objects from TEST                       |23.0 [00:00,    223obj/s]
Building data objects from TEST                       |52.0 [00:00,    258obj/s]
Building data objects from TEST                       |82.0 [00:00,    276obj/s]
Building data objects from TEST                        |110 [00:00,    276obj/s]
Building data objects from TEST                        |168 [00:00,    383obj/s]
Building data objects from TEST                        |226 [00:00,    448obj/s]
Building data objects from TEST                        |287 [00:00,    499obj/s]
Building data objects from TEST                        |344 [00:00,    522obj/s]
Building data objects from TEST                        |410 [00:00,    562obj/s]
Building data objects f

Отправим данные в хранилище

In [28]:
! dvc push TEST.dvc --remote=mygoogledrive

Everything is up to date.                                                       


In [29]:
! dvc add train_val

⠋ Checking graph                                                   ⠋ Checking graph
Adding...                                                                       
!
Building data objects from train_val                  |0.00 [00:00,      ?obj/s]
Building data objects from train_val                  |20.0 [00:00,    197obj/s]
Building data objects from train_val                  |41.0 [00:00,    201obj/s]
Building data objects from train_val                  |67.0 [00:00,    223obj/s]
Building data objects from train_val                  |90.0 [00:00,    220obj/s]
Building data objects from train_val                   |114 [00:00,    225obj/s]
Building data objects from train_val                   |137 [00:00,    221obj/s]
Building data objects from train_val                   |163 [00:00,    230obj/s]
Building data objects from train_val                   |187 [00:00,    226obj/s]
Building data objects from train_val                   |210 [00:00,    226obj/s]
Building data objects f

In [30]:
! dvc push train_val.dvc --remote=mygoogledrive

Everything is up to date.                                                       


Сохраним ярлыки в git
`TODO`

### Плохая практика 3: проведение экспериментов без отслеживания результатов
Достижение лучшх результатов в любой ml-задаче требуют множество экспериментов, каждый из которых дает небольшое улучшение (или ухудшение).

Результаты экспериментов необходимо сравнивать между собой, чтобы оставлять успешные идеи и отбрасывать неудачные.

Простейший (ручной) способ отслеживания экспериментов:
- выполнили эксперимент N, запомнили метрики
- выполнили эксперимент N+1, сравнили метрики с N, запомнили
- выполнили эксперимент N+2, сравнили метрики с N+1, запомнили
- ...

Главная проблема: **после эксперимента не остаётся следов (артефактов)**

Проблемы-следствия:
- нельзя сравнить результаты эксперименты N и (N+10) 
- сложно провести многовариантный, а не бинарный эксперимент
- сложно воспроизвести идею N (если она стала снова актуальной)
- нужно держать в голове, насколько хорощо/плохо сработала идея когда-то в прошлом

**Решение:** Логировать все параметры и результаты эксперимента

### 3.1 Подключаем Weights & Biases
https://docs.wandb.ai/quickstart

1. Устанавливаем wandb: `! pip install wandb`
2. Авторизуемся на `https://wandb.ai/login` (например, через GitHub)
3. Заходим на `https://wandb.ai/settings`, копируем ключ из `API keys`
4. Выполняем `$ wandb login <YOUR API KEY>` 

### 3.2 Проверяем wnb

In [72]:
import wandb

In [73]:
wandb.init(project='hello_world', config={'lr': 0.01, 'foo': 'bar', 'something': True}, name='second')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vashchilkoav. Use `wandb login --relogin` to force relogin


Можно логировать численные величины. Каждое вызов log неявно увеличивает внутренний счётчик шагов.

In [74]:
wandb.log({"train": {"loss": 0.9, "metric": 0.5}, "val": {"loss": 0.4, "acc": 0.8}})

In [75]:
wandb.log({"train": {"loss": 0.8, "metric": 0.5}, "val": {"loss": 0.35, "acc": 0.8}})

In [76]:
wandb.log({"train": {"loss": 0.75, "metric": 0.52}, "val": {"loss": 0.33, "acc": 0.7}})

In [77]:
wandb.log({"train": {"loss": 0.74, "metric": 0.52}, "val": {"loss": 0.32, "acc": 0.7}})

Можно явно указать шаг, к которому относится запись

In [78]:
wandb.log({"train": {"loss": 0.74, "metric": 0.52}, "val": {"loss": 0.32, "acc": 0.7}}, step=20)

Каждый вызов log добавляет аргументы во внутреннее состояние и коммитит **предыдущие** значения.

Можно считать, что каждый вызов .log - это `git commit` старых данных + `git add` новых данных.

In [79]:
wandb.log({"train": {"loss": 0.74, "metric": 0.52}, "val": {"loss": 0.32, "acc": 0.7}}, step=50)

Можно логировать не только численные величины - например, изображения с масками.

In [81]:
from suim_segmentation.data import SuimDataset

test_data = SuimDataset(root=ROOT_PATH / 'data' / 'test', masks_as_color=False)

In [82]:
x_img, y_mask = test_data[2]

In [83]:
SuimDataset.LABEL_COLORS

(('Background(waterbody)', '000'),
 ('Human divers', '001'),
 ('Aquatic plants and sea-grass', '010'),
 ('Wrecks and ruins', '011'),
 ('Robots (AUVs/ROVs/instruments)', '100'),
 ('Reefs and invertebrates', '101'),
 ('Fish and vertebrates', '110'),
 ('Sea-floor and rocks', '111'))

In [84]:
class_labels = dict(
    (num, cls_name) for num, (cls_name, binary_idx) in enumerate(SuimDataset.LABEL_COLORS)
)
mask_img = wandb.Image(x_img.permute(1, 2, 0).numpy(), masks={
  "predictions": {
    "mask_data": y_mask[0].numpy(),
    "class_labels": class_labels
  }
})

In [85]:
wandb.log({"gt_example": mask_img})

Можно добавить ключ-значение в summary

In [86]:
wandb.run.summary['my_key'] = 'my_important_value'

In [87]:
wandb.finish()

my_key,my_important_value


### 3.3 Логируем параметры и результаты эксперимента
### Задачка 3
Дописать код run.py так, чтобы логировались (как минимум):
- Средние train.loss, train.metric, val.loss, val.metric для каждой эпохт
- Средние test.loss, test.metric однократно

Сделать проект публичным, скинуть ссылку на свой wandb отчет

In [ ]:
! python -m suim_segmentation.run --name=baseline --lr=0.03 --num-epochs=1 --batch-size=32 --device=cuda:0 \
    --train-data=/home/alexander/computerScience/phystech/9sem/abbyy/course_cvdl/data/train_val \
    --test-data=/home/alexander/computerScience/phystech/9sem/abbyy/course_cvdl/data/TEST